import Basic Module

In [1]:
import os
import torch
import numpy as np
from pathlib import Path
import time
from datetime import datetime, timedelta
from tqdm import tqdm
import importlib
import json

# Utils import (모듈화)
from utils import create_dataloaders, CDMetrics, get_loss_fn, CDTrainer



/userHome/userhome4/kyoungmin/miniconda3/envs/cd_efficient/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA(GPU) 확인

In [2]:
# 단일 GPU 사용
GPU_ID = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using GPU: {GPU_ID}")
print(f"Device: {DEVICE}")

# 멀티 GPU 사용 
# # GPU_IDS = [0, 1, 2, 3]  # 사용할 GPU 리스트
# os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(map(str, GPU_IDS))
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# USE_MULTI_GPU = len(GPU_IDS) > 1 and torch.cuda.device_count() > 1
# print(f"Using GPUs: {GPU_IDS}")
# print(f"Available GPU count: {torch.cuda.device_count()}")
# if USE_MULTI_GPU:
#     BATCH_SIZE = BATCH_SIZE * len(GPU_IDS)  # 멀티 GPU시 배치 크기 조정
#     print(f"Adjusted batch size for multi-GPU: {BATCH_SIZE}")
# 시드 설정 (재현가능성)

Using GPU: 3
Device: cuda


데이터셋 & 모델 리스트

In [3]:
DATASET_ROOT = "./dataset"  # 심볼릭 링크된 데이터셋 루트 폴더
DATASET_LIST = [
    'LEVIR-CD+',
    'WHU-CD',
    'CLCD',
    'CaBuAr-CD',
    'S2Looking-CD',
    'SEN1Floods11-CD'
]

MODEL_LIST = [
    'A2Net',
    'Changer',
    'Change3D',
    'STRobustNet',
    'USSFC-Net'
    # 'ChangeMamba',
    # 'CDMamba',
    # 'ChangeCLIP',
    # 'EATDER'
]

시드 설정

In [4]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

##### 고정 메트릭 설정

In [5]:
# 이미지 설정
IMG_SIZE = 256 
IN_CHANNELS = 3  # RGB
OUT_CHANNELS = 1  # Binary change detection

# 학습 설정
BATCH_SIZE = 64
NUM_WORKERS = 4
MAX_ITERATIONS = 100000  # 데이터셋 크기와 무관하게 고정


##### 모델 & 데이터셋 설정

In [6]:
# 실험할 데이터셋 선택
test_dataset = 'LEVIR-CD+'  # LEVIR-CD+
# test_dataset = DATASET_LIST[0]  # LEVIR-CD+

# 실험할 모델 선택
test_model = 'A2Net'

# True: 최소 구현, False: 전체 구현
use_base = True  

print(f"Test dataset: {test_dataset}")
if test_model not in MODEL_LIST:
    raise ValueError(f"Model {test_model} not in MODEL_LIST. Choose from: {MODEL_LIST}")
print(f"Test model: {test_model}")
print(f"Using base version: {use_base}")

Test dataset: LEVIR-CD+
Test model: A2Net
Using base version: True


모델 동적 import

In [7]:
def get_model_class(model_name, use_base=False):
    """모델 동적 import - 자동 경로 생성"""
    
    # 모델명을 소문자로 변환
    model_name_lower = model_name.lower()
    
    # base/full에 따른 경로 및 클래스명 생성
    if use_base:
        module_path = f'models.{model_name_lower}_base'
        class_name = f'{model_name}Base'
    else:
        module_path = f'models.{model_name_lower}'
        class_name = model_name
    
    try:
        module = importlib.import_module(module_path)
        return getattr(module, class_name)
    except (ImportError, AttributeError) as e:
        if use_base:
            raise ImportError(f"{model_name} base version not found at {module_path}.{class_name}: {e}")
        else:
            # Full 버전이 없으면 base로 폴백
            print(f"Full version not found ({module_path}.{class_name}), falling back to base version")
            return get_model_class(model_name, use_base=True)

ModelClass = get_model_class(test_model, use_base)
print(f"Loaded: {ModelClass.__name__}")

Loaded: A2NetBase


In [8]:
from configs import get_model_config

model_config = get_model_config(test_model)

optimizer = model_config['optimizer']
learning_rate = model_config['learning_rate']
weight_decay = model_config['weight_decay']
betas = model_config['betas']
eps = model_config['eps']
scheduler = model_config['scheduler']
momentum = model_config['momentum']

print(f"Model configurations:")
print(f"  Optimizer: {optimizer}")
print(f"  Learning rate: {learning_rate}")
print(f"  Weight decay: {weight_decay}")
if betas:
    print(f"  Betas: {betas}")
if momentum:
    print(f"  Momentum: {momentum}")
print(f"  Scheduler: {scheduler}")


Model configurations:
  Optimizer: adam
  Learning rate: 0.0005
  Weight decay: 0.0001
  Betas: (0.9, 0.99)
  Scheduler: poly


옵티마이저 생성 함수

In [9]:
def get_optimizer(model, config):
    """모델 설정에 따른 옵티마이저 생성"""
    if config['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(
            model.parameters(), 
            lr=config['learning_rate'],
            betas=config['betas'],
            eps=config['eps'],
            weight_decay=config['weight_decay']
        )
    elif config['optimizer'] == 'adamw':
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=config['learning_rate'],
            betas=config['betas'],
            eps=config['eps'],
            weight_decay=config['weight_decay']
        )
    elif config['optimizer'] == 'sgd':
        optimizer = torch.optim.SGD(
            model.parameters(),
            lr=config['learning_rate'],
            momentum=config['momentum'],
            weight_decay=config['weight_decay']
        )
    else:
        raise ValueError(f"Unknown optimizer: {config['optimizer']}")
    
    return optimizer

실험 폴더 설정

In [10]:
test_path = f"experiments/{test_dataset}"
test_dir = Path(f"{test_path}")
test_dir.mkdir(parents=True, exist_ok=True)

model_dir = Path(f"{test_path}/{test_model}")
model_dir.mkdir(parents=True, exist_ok=True)

checkpoint_dir = model_dir / "checkpoints"
checkpoint_dir.mkdir(exist_ok=True)

print(f"Experiment directory: {model_dir}")

Experiment directory: experiments/LEVIR-CD+/A2Net


Iteration 계산 
MAX_ITERATIONS = 100000을 기준으로 epoch 수 계산


In [11]:
dataset_path = Path(DATASET_ROOT) / test_dataset
print(dataset_path)
if dataset_path.exists():
    print(f"Dataset found: {dataset_path}")
    splits = ['train', 'val', 'test']
    dataset_info = {}
    
    for split in splits:
        split_path = dataset_path / split
        if split_path.exists():
            img_count = len(list((split_path / 't1').glob('*')))
            dataset_info[split] = img_count
            print(f"  {split}: {img_count} images")
    
    # Epoch 수 계산 (MAX_ITERATIONS 기준)
    if 'train' in dataset_info:
        train_samples = dataset_info['train']
        iterations_per_epoch = train_samples // BATCH_SIZE
        EPOCHS = MAX_ITERATIONS // iterations_per_epoch
        
        print(f"\nTraining iterations info:")
        print(f"  Train samples: {train_samples}")
        print(f"  Iterations per epoch: {iterations_per_epoch}")
        print(f"  Total epochs: {EPOCHS}")
        print(f"  Total iterations: {EPOCHS * iterations_per_epoch}")
else:
    print(f"Dataset not found: {dataset_path}")
    raise FileNotFoundError(f"Dataset {test_dataset} not found at {dataset_path}")

dataset/LEVIR-CD+
Dataset found: dataset/LEVIR-CD+
  train: 10192 images
  val: 1568 images
  test: 4000 images

Training iterations info:
  Train samples: 10192
  Iterations per epoch: 159
  Total epochs: 628
  Total iterations: 99852


데이터로더 생성

In [12]:
train_loader, val_loader, test_loader = create_dataloaders(
    root_dir=DATASET_ROOT,
    dataset_name=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)


Loaded 10192 images from LEVIR-CD+/train
Loaded 1568 images from LEVIR-CD+/val
Loaded 4000 images from LEVIR-CD+/test


모델 학습 및 검증

In [ ]:
model = ModelClass(num_classes=1).to(DEVICE)
criterion = get_loss_fn('bce_dice')

# 옵티마이저
if optimizer == 'adam':
    opt = torch.optim.Adam(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
elif optimizer == 'adamw':
    opt = torch.optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )

# 스케줄러
sched = None
if scheduler == 'cosine':
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)

# %% Trainer 생성 및 학습
trainer = CDTrainer(
    model=model,
    optimizer=opt,
    criterion=criterion,
    device=DEVICE,
    checkpoint_dir=checkpoint_dir,
    scheduler=sched
)

# 학습 실행
trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=EPOCHS,
    val_interval=1,
    save_interval=50
)

# %% 테스트 및 속도 측정
# 테스트
test_metrics = trainer.test(test_loader)

# 추론 속도
speed_metrics = trainer.measure_inference_speed(test_loader)

# 결과 저장
trainer.save_results(model_dir, test_model, test_dataset)

# %% 모델 파라미터 수
total_params = sum(p.numel() for p in model.parameters())
print(f"\nModel Parameters: {total_params:,}")

# 최종 결과
final_results = {
    'model': test_model,
    'base_version': use_base,
    'dataset': test_dataset,
    'test_metrics': test_metrics,
    'speed_metrics': speed_metrics,
    'parameters': total_params
}

with open(model_dir / 'final_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("\n✓ All completed!")

In [14]:

print("\n" + "="*60)
print("Inference & Speed Measurement")
print("="*60)

# %% Best 모델 로드
best_model_path = checkpoint_dir / 'best_model.pth'

if best_model_path.exists():
    checkpoint = torch.load(best_model_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"\n✓ Loaded best model from epoch {checkpoint['epoch']}")
    print(f"  Best F1: {checkpoint.get('best_f1', 0):.4f}")
    print(f"  Best IoU: {checkpoint.get('best_iou', 0):.4f}")
else:
    print("\n⚠ Best model not found, using current model state")

model.eval()

# %% 전체 테스트셋 추론
print("\n" + "-"*60)
print("Testing on full test set...")
print("-"*60)

test_metrics = CDMetrics()
test_loss = 0

# 추론 시간 측정용
inference_times = []
total_pixels = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        img1 = batch['img1'].to(DEVICE)
        img2 = batch['img2'].to(DEVICE)
        label = batch['label'].to(DEVICE)
        
        # Loss 계산
        output = model(img1, img2)
        loss = criterion(output, label)
        test_loss += loss.item()
        
        # 메트릭 업데이트
        test_metrics.update(output, label)
        
        # 추론 시간 측정 (개별 샘플)
        for i in range(img1.size(0)):
            if DEVICE.type == 'cuda':
                torch.cuda.synchronize()
            
            start_time = time.time()
            _ = model(img1[i:i+1], img2[i:i+1])
            
            if DEVICE.type == 'cuda':
                torch.cuda.synchronize()
            
            inference_times.append(time.time() - start_time)
            total_pixels += img1.size(-2) * img1.size(-1)

avg_test_loss = test_loss / len(test_loader)
test_results = test_metrics.get_metrics()

print(f"\n{'Test Results':^60}")
print("="*60)
print(f"  Loss:      {avg_test_loss:.4f}")
print(f"  F1 Score:  {test_results['f1']:.4f}")
print(f"  IoU:       {test_results['iou']:.4f}")
print(f"  Precision: {test_results['precision']:.4f}")
print(f"  Recall:    {test_results['recall']:.4f}")
print(f"  OA:        {test_results['oa']:.4f}")
print(f"  Kappa:     {test_results['kappa']:.4f}")
print("="*60)

# %% 추론 속도 분석
print(f"\n{'Inference Speed Analysis':^60}")
print("="*60)

inference_times = np.array(inference_times)
avg_time = np.mean(inference_times)
std_time = np.std(inference_times)
min_time = np.min(inference_times)
max_time = np.max(inference_times)

# FPS 계산
fps = 1.0 / avg_time

# 이미지당 처리 시간
time_per_image_ms = avg_time * 1000

# 픽셀당 처리 시간
avg_pixels_per_image = total_pixels / len(inference_times)
time_per_megapixel = (avg_time / avg_pixels_per_image) * 1e6 * 1000  # ms

print(f"  Total samples:     {len(inference_times)}")
print(f"  Average time:      {time_per_image_ms:.2f} ms/image")
print(f"  Std deviation:     {std_time*1000:.2f} ms")
print(f"  Min time:          {min_time*1000:.2f} ms")
print(f"  Max time:          {max_time*1000:.2f} ms")
print(f"  FPS:               {fps:.2f} images/sec")
print(f"  Time/Megapixel:    {time_per_megapixel:.2f} ms/MP")
print("="*60)

# %% 배치 단위 추론 속도 (더 정확한 측정)
print(f"\n{'Batch Inference Speed':^60}")
print("="*60)

# Warmup
warmup_batch = next(iter(test_loader))
img1_warmup = warmup_batch['img1'].to(DEVICE)
img2_warmup = warmup_batch['img2'].to(DEVICE)

for _ in range(5):
    with torch.no_grad():
        _ = model(img1_warmup, img2_warmup)

if DEVICE.type == 'cuda':
    torch.cuda.synchronize()

# 실제 측정 (10 배치)
num_measure_batches = min(10, len(test_loader))
batch_times = []

test_iter = iter(test_loader)
for _ in range(num_measure_batches):
    batch = next(test_iter)
    img1 = batch['img1'].to(DEVICE)
    img2 = batch['img2'].to(DEVICE)
    
    if DEVICE.type == 'cuda':
        torch.cuda.synchronize()
    
    start = time.time()
    
    with torch.no_grad():
        _ = model(img1, img2)
    
    if DEVICE.type == 'cuda':
        torch.cuda.synchronize()
    
    batch_times.append(time.time() - start)

avg_batch_time = np.mean(batch_times)
batch_fps = BATCH_SIZE / avg_batch_time
single_image_time = avg_batch_time / BATCH_SIZE

print(f"  Batch size:        {BATCH_SIZE}")
print(f"  Batches measured:  {num_measure_batches}")
print(f"  Avg batch time:    {avg_batch_time*1000:.2f} ms")
print(f"  Batch FPS:         {batch_fps:.2f} images/sec")
print(f"  Per image:         {single_image_time*1000:.2f} ms")
print("="*60)

# %% 모델 복잡도 정보
print(f"\n{'Model Complexity':^60}")
print("="*60)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"  Total parameters:      {total_params:,}")
print(f"  Trainable parameters:  {trainable_params:,}")
print(f"  Model size:            {total_params * 4 / 1024 / 1024:.2f} MB (FP32)")
print("="*60)

# %% 결과 저장
results_dict = {
    'model': test_model,
    'dataset': test_dataset,
    'test_metrics': {
        'loss': avg_test_loss,
        'f1': test_results['f1'],
        'iou': test_results['iou'],
        'precision': test_results['precision'],
        'recall': test_results['recall'],
        'oa': test_results['oa'],
        'kappa': test_results['kappa']
    },
    'inference_speed': {
        'avg_time_ms': float(time_per_image_ms),
        'std_time_ms': float(std_time * 1000),
        'fps': float(fps),
        'batch_time_ms': float(avg_batch_time * 1000),
        'batch_fps': float(batch_fps)
    },
    'model_complexity': {
        'total_params': int(total_params),
        'trainable_params': int(trainable_params),
        'model_size_mb': float(total_params * 4 / 1024 / 1024)
    }
}

# JSON 저장
import json
results_path = model_dir / 'test_results.json'
with open(results_path, 'w') as f:
    json.dump(results_dict, f, indent=2)

print(f"\n✓ Results saved to {results_path}")

print("\n" + "="*60)
print("Inference Complete!")
print("="*60)


Inference & Speed Measurement


NameError: name 'model' is not defined